# Gesture Recognition
## Project by V S S Anirudh Sharma
In this project, we will build a 3D Conv/ RNN based model that will be able to predict the 5 gestures correctly.

In [1]:
import numpy as np
import os
# from scipy.ndimage import imread, imresize
from PIL import Image
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip "/content/drive/MyDrive/Gesture Recognition/Project_data.zip"

Streaming output truncated to the last 5000 lines.
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00050.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00052.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00054.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00056.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00058.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00060.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00062.png  
   creating: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/
 extracting: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/WIN_20180926_17_35_12_Pro_00001.png  
 extracting: Project_data/t

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [5]:
train_doc = np.random.permutation(open('/content/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/Project_data/val.csv').readlines())
batch_size = 32#experiment with the batch size
dim_x = dim_y = 100
selected_frames = range(0,30,2) #create a list of image numbers you want to use for a particular video

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [23]:
def normalize_frame(image):
  return (image-np.mean(image))/np.std(image)

In [24]:
def generator(source_path, folder_list, batch_size, dim_x = dim_x,dim_y=dim_y,selected_frames = selected_frames):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = selected_frames
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size
        remaining_images = len(t)%batch_size
        x = len(img_idx)
        y,z = dim_x,dim_y

        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in

                    image = Image.open(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    width, height = image.size
                    new_width = new_height = min(width, height)
                    left = (width - new_width)/2
                    top = (height - new_height)/2
                    right = (width + new_width)/2
                    bottom = (height + new_height)/2

                    # Crop the center of the image
                    image_cropped = image.crop((left, top, right, bottom))
                    image_resized = image_cropped.resize((dim_x,dim_y))

                    image_array = np.asarray(image_resized).astype(np.float32)

                    # batch_data[folder,idx,:,:,0] = image_array[:,:,0]/255#normalise and feed in the image
                    # batch_data[folder,idx,:,:,1] = image_array[:,:,1]/255#normalise and feed in the image
                    # batch_data[folder,idx,:,:,2] = image_array[:,:,2]/255#normalise and feed in the image

                    batch_data[folder,idx,:,:,0] = normalize_frame(image_array[:,:,0])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = normalize_frame(image_array[:,:,1])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalize_frame(image_array[:,:,2])#normalise and feed in the image


                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        batch = num_batches
        batch_data = np.zeros((remaining_images,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((remaining_images,5)) # batch_labels is the one hot representation of the output
        for folder in range(remaining_images): # iterate over the batch_size
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in

                image = Image.open(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                #crop the images and resize them. Note that the images are of 2 different shape
                #and the conv3D will throw error if the inputs in a batch have different shapes

                width, height = image.size
                new_width = new_height = min(width, height)
                left = (width - new_width)/2
                top = (height - new_height)/2
                right = (width + new_width)/2
                bottom = (height + new_height)/2

                # Crop the center of the image
                image_cropped = image.crop((left, top, right, bottom))
                image_resized = image_cropped.resize((dim_x,dim_y))

                image_array = np.asarray(image_resized).astype(np.float32)

                # batch_data[folder,idx,:,:,0] = image_array[:,:,0]/255#normalise and feed in the image
                # batch_data[folder,idx,:,:,1] = image_array[:,:,1]/255#normalise and feed in the image
                # batch_data[folder,idx,:,:,2] = image_array[:,:,2]/255#normalise and feed in the image

                batch_data[folder,idx,:,:,0] = normalize_frame(image_array[:,:,0])#normalise and feed in the image
                batch_data[folder,idx,:,:,1] = normalize_frame(image_array[:,:,1])#normalise and feed in the image
                batch_data[folder,idx,:,:,2] = normalize_frame(image_array[:,:,2])#normalise and feed in the image

            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
        yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do



Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/Project_data/train'
val_path = '/content/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [78]:
from keras.models import Sequential, Model

#from keras.layers import Dense, GRU, Flatten, TimeDistributed, Bidirectional, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D, GlobalAveragePooling3D, ConvLSTM2D
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, Bidirectional, GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, Conv3D, MaxPooling2D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

# EXPERIMENT 1: Simple Conv3D
with only one layer

In [9]:
x = len(selected_frames)
y,z = dim_x,dim_y
input_shape = (x,y,z, 3)


model0 = Sequential()
model0.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model0.add(MaxPooling3D(pool_size=2))

model0.add(Flatten())
model0.add(Dense(128, activation='relu'))
model0.add(Dense(5, activation='softmax'))

In [10]:
optimiser = optimizers.Adam()#write your optimizer
model0.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model0.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 13, 98, 98, 32)    2624      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 6, 49, 49, 32)    0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 460992)            0         
                                                                 
 dense (Dense)               (None, 128)               59007104  
                                                                 
 dense_1 (Dense)             (None, 5)                 645       
                                                                 
Total params: 59,010,373
Trainable params: 59,010,373
Non-trainable params: 0
____________________________________________

In [11]:
model_name = 'model_init_0' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [12]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [14]:
model0.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-14-c029bc73bd72>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model0.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /content/Project_data/train ; batch size = 32
Epoch 1/10
21/21 [==============================] - ETA: 0s - loss: 12.6977 - categorical_accuracy: 0.2293Source path =  /content/Project_data/val ; batch size = 32

Epoch 1: saving model to model_init_0_2023-07-0205_46_55.002597/model-00001-12.69773-0.22926-1.50754-0.35000.h5
21/21 [==============================] - 79s 3s/step - loss: 12.6977 - categorical_accuracy: 0.2293 - val_loss: 1.5075 - val_categorical_accuracy: 0.3500 - lr: 0.0010
Epoch 2/10
21/21 [==============================] - ETA: 0s - loss: 1.4868 - categorical_accuracy: 0.3605
Epoch 2: saving model to model_init_0_2023-07-0205_46_55.002597/model-00002-1.48680-0.36048-1.48399-0.42000.h5
21/21 [==============================] - 43s 2s/step - loss: 1.4868 - categorical_accuracy: 0.3605 - val_loss: 1.4840 - val_categorical_accuracy: 0.4200 - lr: 0.0010
Epoch 3/10
21/21 [==============================] - ETA: 0s - loss: 1.1518 - categorical_accuracy: 0.5400
Epoch

## Observations:
1. The models best behaviour is 99% training and 65% validation accuracy
2. Too much overfitting

## TODO:
1. Reduce overfitting
2. Introduce dropout layer

# EXPERIMENT 2: Simple COnv3D
with only one layer with dropouts

In [16]:
x = len(selected_frames)
y,z = dim_x,dim_y
input_shape = (x,y,z, 3)


model05 = Sequential()
model05.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model05.add(MaxPooling3D(pool_size=2))
model05.add(Dropout(0.5))

model05.add(Flatten())
model05.add(Dense(128, activation='relu'))
model05.add(Dense(5, activation='softmax'))

In [18]:
optimiser = optimizers.Adam()#write your optimizer
model05.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model05.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_1 (Conv3D)           (None, 13, 98, 98, 32)    2624      
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 6, 49, 49, 32)    0         
 3D)                                                             
                                                                 
 dropout (Dropout)           (None, 6, 49, 49, 32)     0         
                                                                 
 flatten_1 (Flatten)         (None, 460992)            0         
                                                                 
 dense_2 (Dense)             (None, 128)               59007104  
                                                                 
 dense_3 (Dense)             (None, 5)                 645       
                                                      

In [19]:
model_name = 'model_init_0_5' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [20]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [21]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [22]:
model05.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-22-592150c29ec6>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model05.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /content/Project_data/train ; batch size = 32
Epoch 1/10
21/21 [==============================] - ETA: 0s - loss: 13.0509 - categorical_accuracy: 0.2549Source path =  /content/Project_data/val ; batch size = 32

Epoch 1: saving model to model_init_0_5_2023-07-0205_46_55.002597/model-00001-13.05088-0.25490-1.48469-0.25000.h5
21/21 [==============================] - 43s 2s/step - loss: 13.0509 - categorical_accuracy: 0.2549 - val_loss: 1.4847 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 2/10
21/21 [==============================] - ETA: 0s - loss: 1.4332 - categorical_accuracy: 0.2594
Epoch 2: saving model to model_init_0_5_2023-07-0205_46_55.002597/model-00002-1.43324-0.25943-1.37299-0.37000.h5
21/21 [==============================] - 54s 3s/step - loss: 1.4332 - categorical_accuracy: 0.2594 - val_loss: 1.3730 - val_categorical_accuracy: 0.3700 - lr: 0.0010
Epoch 3/10
21/21 [==============================] - ETA: 0s - loss: 0.9418 - categorical_accuracy: 0.6456
E

## Observations:
1. The models best behaviour is 100% training and 57% validation accuracy
2. We are worse off

## TODO:
1. Add layer to improve performance.

# EXPERIMENT 3: Conv3D
We start with 2 layers layers

In [25]:
#write your model here
x = len(selected_frames)
y,z = dim_x,dim_y
input_shape = (x,y,z, 3)


model1 = Sequential()
model1.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model1.add(MaxPooling3D(pool_size=2))

model1.add(Conv3D(64, kernel_size=3, activation='relu'))
model1.add(MaxPooling3D(pool_size=2))

model1.add(Flatten())
model1.add(Dense(256, activation='relu'))
model1.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [26]:
optimiser = optimizers.Adam()#write your optimizer
model1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model1.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 13, 98, 98, 32)    2624      
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 6, 49, 49, 32)    0         
 3D)                                                             
                                                                 
 conv3d_5 (Conv3D)           (None, 4, 47, 47, 64)     55360     
                                                                 
 max_pooling3d_5 (MaxPooling  (None, 2, 23, 23, 64)    0         
 3D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 67712)             0         
                                                                 
 dense_4 (Dense)             (None, 256)              

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [27]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [28]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [29]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [30]:
model1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-30-3dbd666228bb>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /content/Project_data/train ; batch size = 32
Epoch 1/10
21/21 [==============================] - ETA: 0s - loss: 1.9002 - categorical_accuracy: 0.2519Source path =  /content/Project_data/val ; batch size = 32

Epoch 1: saving model to model_init_2023-07-0205_03_51.402388/model-00001-1.90017-0.25189-1.35562-0.39000.h5
21/21 [==============================] - 40s 2s/step - loss: 1.9002 - categorical_accuracy: 0.2519 - val_loss: 1.3556 - val_categorical_accuracy: 0.3900 - lr: 0.0010
Epoch 2/10
21/21 [==============================] - ETA: 0s - loss: 1.2004 - categorical_accuracy: 0.5173
Epoch 2: saving model to model_init_2023-07-0205_03_51.402388/model-00002-1.20043-0.51735-1.10210-0.56000.h5
21/21 [==============================] - 44s 2s/step - loss: 1.2004 - categorical_accuracy: 0.5173 - val_loss: 1.1021 - val_categorical_accuracy: 0.5600 - lr: 0.0010
Epoch 3/10
21/21 [==============================] - ETA: 0s - loss: 0.7219 - categorical_accuracy: 0.7285
Epoch 3: sav

## Observations
1. We have obtained a very good model in the 7th epoch, with 99.7% training accuracy and 74% validation accuracy.
2. We observe overfitting since training accuracy is much higher than validation accuracy

## TODO:
Let's add drop outs

## Experiment 4: Conv3D: Add dropouts

In [25]:
model2 = Sequential()
model2.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model2.add(MaxPooling3D(pool_size=2))
model2.add(Dropout(0.1))
model2.add(Conv3D(64, kernel_size=3, activation='relu'))
model2.add(MaxPooling3D(pool_size=2))
model2.add(Dropout(0.1))


model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dense(5, activation='softmax'))

In [26]:
optimiser = optimizers.Adam()#write your optimizer
model2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_2 (Conv3D)           (None, 13, 98, 98, 32)    2624      
                                                                 
 max_pooling3d_2 (MaxPooling  (None, 6, 49, 49, 32)    0         
 3D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 6, 49, 49, 32)     0         
                                                                 
 conv3d_3 (Conv3D)           (None, 4, 47, 47, 64)     55360     
                                                                 
 max_pooling3d_3 (MaxPooling  (None, 2, 23, 23, 64)    0         
 3D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 2, 23, 23, 64)    

In [27]:
model_name = 'model_init_2' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [28]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [32]:
num_epochs = 5
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-32-244470d6f2b1>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/5
21/21 [==============================] - ETA: 0s - loss: 0.0043 - categorical_accuracy: 1.0000
Epoch 1: saving model to model_init_2_2023-07-0205_46_55.002597/model-00001-0.00433-1.00000-3.37083-0.49000.h5
21/21 [==============================] - 38s 2s/step - loss: 0.0043 - categorical_accuracy: 1.0000 - val_loss: 3.3708 - val_categorical_accuracy: 0.4900 - lr: 0.0010
Epoch 2/5
21/21 [==============================] - ETA: 0s - loss: 0.0039 - categorical_accuracy: 1.0000
Epoch 2: saving model to model_init_2_2023-07-0205_46_55.002597/model-00002-0.00387-1.00000-1.87561-0.65000.h5
21/21 [==============================] - 40s 2s/step - loss: 0.0039 - categorical_accuracy: 1.0000 - val_loss: 1.8756 - val_categorical_accuracy: 0.6500 - lr: 0.0010
Epoch 3/5
21/21 [==============================] - ETA: 0s - loss: 0.0019 - categorical_accuracy: 1.0000
Epoch 3: saving model to model_init_2_2023-07-0205_46_55.002597/model-00003-0.00193-1.00000-2.37033-0.60000.h5
21/21 [=============

## Observations:
1. Best result: 100% train and 65% test accuracy
2. We are worse off

## Hypothesis
1. Increasing model complexity while introducing regularizarion must rightly caputure the patter

## TODO:
Inrease model complexity

## Experiment 5: More layers

In [36]:
model3 = Sequential()
model3.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model3.add(Conv3D(32, kernel_size=3, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model3.add(MaxPooling3D(pool_size=2))
model3.add(Dropout(0.25))

model3.add(Conv3D(64, kernel_size=3, activation='relu'))
model3.add(Conv3D(64, kernel_size=3, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model3.add(MaxPooling3D(pool_size=(1,2,2)))
model3.add(Dropout(0.25))


model3.add(Flatten())
model3.add(Dense(256, activation='relu'),)
model3.add(Dense(5, activation='softmax'))

In [39]:
optimiser = optimizers.Adam()#write your optimizer
model3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model3.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_16 (Conv3D)          (None, 13, 98, 98, 32)    2624      
                                                                 
 conv3d_17 (Conv3D)          (None, 11, 96, 96, 32)    27680     
                                                                 
 max_pooling3d_10 (MaxPoolin  (None, 5, 48, 48, 32)    0         
 g3D)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 5, 48, 48, 32)     0         
                                                                 
 conv3d_18 (Conv3D)          (None, 3, 46, 46, 64)     55360     
                                                                 
 conv3d_19 (Conv3D)          (None, 1, 44, 44, 64)     110656    
                                                      

In [40]:
model_name = 'model_init_3' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [41]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [44]:
num_epochs = 10
model3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-44-2bc231e6e589>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/10
21/21 [==============================] - ETA: 0s - loss: 0.2987 - categorical_accuracy: 0.9246
Epoch 1: saving model to model_init_3_2023-07-0205_46_55.002597/model-00001-0.29873-0.92459-1.60480-0.62000.h5
21/21 [==============================] - 41s 2s/step - loss: 0.2987 - categorical_accuracy: 0.9246 - val_loss: 1.6048 - val_categorical_accuracy: 0.6200 - lr: 0.0010
Epoch 2/10
21/21 [==============================] - ETA: 0s - loss: 0.3381 - categorical_accuracy: 0.9050
Epoch 2: saving model to model_init_3_2023-07-0205_46_55.002597/model-00002-0.33813-0.90498-1.50330-0.56000.h5
21/21 [==============================] - 38s 2s/step - loss: 0.3381 - categorical_accuracy: 0.9050 - val_loss: 1.5033 - val_categorical_accuracy: 0.5600 - lr: 0.0010
Epoch 3/10
21/21 [==============================] - ETA: 0s - loss: 0.2738 - categorical_accuracy: 0.9246
Epoch 3: saving model to model_init_3_2023-07-0205_46_55.002597/model-00003-0.27385-0.92459-1.58712-0.62000.h5
21/21 [==========

## Observation
1. Best model has 93% train and 67% val accuracy
2. Good but not the best model we had using 3D convolutions

### LET US VENTURE INTO RNN BASED MODELS

## Experiment 6: GRU with Dropouts and L2 Regularizations

In [72]:
model5 = Sequential()
model5.add(TimeDistributed(Conv2D(32, (3,3), activation='relu'), input_shape=input_shape))
model5.add(TimeDistributed(MaxPooling2D((2,2))))
model5.add(Dropout(0.5))

model5.add(TimeDistributed(Flatten()))
# model5.add(TimeDistributed(Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01))))
# model5.add(Dropout(0.5))

model5.add(GRU(64,kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model5.add(BatchNormalization())
model5.add(Dense(5, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(0.01)))

In [73]:
optimiser = optimizers.Adam()#write your optimizer
model5.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model5.summary())

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_66 (TimeDi  (None, 15, 98, 98, 32)   896       
 stributed)                                                      
                                                                 
 time_distributed_67 (TimeDi  (None, 15, 49, 49, 32)   0         
 stributed)                                                      
                                                                 
 dropout_25 (Dropout)        (None, 15, 49, 49, 32)    0         
                                                                 
 time_distributed_68 (TimeDi  (None, 15, 76832)        0         
 stributed)                                                      
                                                                 
 gru_10 (GRU)                (None, 64)                14764416  
                                                     

In [74]:
model_name = 'model_init_GRU2' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [77]:
num_epochs = 10
model5.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-77-311016790bbf>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model5.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/10
21/21 [==============================] - ETA: 0s - loss: 0.6961 - categorical_accuracy: 0.9608
Epoch 1: saving model to model_init_GRU2_2023-07-0205_46_55.002597/model-00001-0.69611-0.96078-2.41789-0.53000.h5
21/21 [==============================] - 41s 2s/step - loss: 0.6961 - categorical_accuracy: 0.9608 - val_loss: 2.4179 - val_categorical_accuracy: 0.5300 - lr: 0.0010
Epoch 2/10
21/21 [==============================] - ETA: 0s - loss: 0.6228 - categorical_accuracy: 0.9834
Epoch 2: saving model to model_init_GRU2_2023-07-0205_46_55.002597/model-00002-0.62280-0.98341-2.29036-0.60000.h5
21/21 [==============================] - 46s 2s/step - loss: 0.6228 - categorical_accuracy: 0.9834 - val_loss: 2.2904 - val_categorical_accuracy: 0.6000 - lr: 0.0010
Epoch 3/10
21/21 [==============================] - ETA: 0s - loss: 0.5473 - categorical_accuracy: 0.9819
Epoch 3: saving model to model_init_GRU2_2023-07-0205_46_55.002597/model-00003-0.54734-0.98190-2.50146-0.51000.h5
21/21 [=

## Observation
1. ~60% val and ~98% training accuracy
2. Overfitting

## TODO
1. Reduce overfitting

## Hypothesis
1. Reduce model layers will reduce overfitting
2. Adding dropout at GRU layer will reduce overfitting

## Experiment 7: Simple GRU with dropout on GRU layer

In [80]:
model6 = Sequential()
model6.add(TimeDistributed(Conv2D(32, (3,3), activation='relu'), input_shape=input_shape))
model6.add(TimeDistributed(MaxPooling2D((2,2))))
model6.add(Dropout(0.5))

# model6.add(TimeDistributed(Conv2D(64, (3,3), activation='relu'), input_shape=input_shape))
# model6.add(TimeDistributed(MaxPooling2D((2,2))))
# model6.add(Dropout(0.5))

model6.add(TimeDistributed(GlobalAveragePooling2D()))
# model5.add(TimeDistributed(Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01))))
# model5.add(Dropout(0.5))

model6.add(GRU(64,kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model6.add(Dropout(0.5))
model6.add(Dense(5, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(0.01)))

In [81]:
optimiser = optimizers.Adam()#write your optimizer
model6.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model6.summary())

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_71 (TimeDi  (None, 15, 98, 98, 32)   896       
 stributed)                                                      
                                                                 
 time_distributed_72 (TimeDi  (None, 15, 49, 49, 32)   0         
 stributed)                                                      
                                                                 
 dropout_27 (Dropout)        (None, 15, 49, 49, 32)    0         
                                                                 
 time_distributed_73 (TimeDi  (None, 15, 32)           0         
 stributed)                                                      
                                                                 
 gru_11 (GRU)                (None, 64)                18816     
                                                     

In [82]:
model_name = 'model_init_GRU3' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [84]:
num_epochs = 10
model6.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-84-dea9432a0791>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model6.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/10
21/21 [==============================] - ETA: 0s - loss: 2.1730 - categorical_accuracy: 0.1900
Epoch 1: saving model to model_init_GRU3_2023-07-0205_46_55.002597/model-00001-2.17297-0.19005-2.06170-0.26000.h5
21/21 [==============================] - 41s 2s/step - loss: 2.1730 - categorical_accuracy: 0.1900 - val_loss: 2.0617 - val_categorical_accuracy: 0.2600 - lr: 0.0010
Epoch 2/10
21/21 [==============================] - ETA: 0s - loss: 2.0089 - categorical_accuracy: 0.2066
Epoch 2: saving model to model_init_GRU3_2023-07-0205_46_55.002597/model-00002-2.00888-0.20664-1.93905-0.18000.h5
21/21 [==============================] - 37s 2s/step - loss: 2.0089 - categorical_accuracy: 0.2066 - val_loss: 1.9391 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 3/10
21/21 [==============================] - ETA: 0s - loss: 1.9009 - categorical_accuracy: 0.1855
Epoch 3: saving model to model_init_GRU3_2023-07-0205_46_55.002597/model-00003-1.90088-0.18552-1.84778-0.18000.h5
21/21 [=

## observation
1. No more overfitting. This time we have underfitting
2. We need to reduce dropout percentage of GRU layer


## Experiment 8: GRU with reduced dropout on GRU layer

In [85]:
model7 = Sequential()
model7.add(TimeDistributed(Conv2D(32, (3,3), activation='relu'), input_shape=input_shape))
model7.add(TimeDistributed(MaxPooling2D((2,2))))
model7.add(Dropout(0.5))

# model6.add(TimeDistributed(Conv2D(64, (3,3), activation='relu'), input_shape=input_shape))
# model6.add(TimeDistributed(MaxPooling2D((2,2))))
# model6.add(Dropout(0.5))

model7.add(TimeDistributed(GlobalAveragePooling2D()))
# model5.add(TimeDistributed(Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01))))
# model5.add(Dropout(0.5))

model7.add(GRU(64,kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model7.add(Dropout(0.2))
model7.add(Dense(5, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(0.01)))

In [86]:
optimiser = optimizers.Adam()#write your optimizer
model7.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model7.summary())

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_74 (TimeDi  (None, 15, 98, 98, 32)   896       
 stributed)                                                      
                                                                 
 time_distributed_75 (TimeDi  (None, 15, 49, 49, 32)   0         
 stributed)                                                      
                                                                 
 dropout_29 (Dropout)        (None, 15, 49, 49, 32)    0         
                                                                 
 time_distributed_76 (TimeDi  (None, 15, 32)           0         
 stributed)                                                      
                                                                 
 gru_12 (GRU)                (None, 64)                18816     
                                                     

In [87]:
model_name = 'model_init_GRU4' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [88]:
num_epochs = 10
model7.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-88-5912b217a743>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model7.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/10
21/21 [==============================] - ETA: 0s - loss: 2.1679 - categorical_accuracy: 0.1991
Epoch 1: saving model to model_init_GRU4_2023-07-0205_46_55.002597/model-00001-2.16794-0.19910-2.06415-0.23000.h5
21/21 [==============================] - 39s 2s/step - loss: 2.1679 - categorical_accuracy: 0.1991 - val_loss: 2.0641 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/10
21/21 [==============================] - ETA: 0s - loss: 2.0004 - categorical_accuracy: 0.2006
Epoch 2: saving model to model_init_GRU4_2023-07-0205_46_55.002597/model-00002-2.00036-0.20060-1.93447-0.17000.h5
21/21 [==============================] - 42s 2s/step - loss: 2.0004 - categorical_accuracy: 0.2006 - val_loss: 1.9345 - val_categorical_accuracy: 0.1700 - lr: 0.0010
Epoch 3/10
21/21 [==============================] - ETA: 0s - loss: 1.8950 - categorical_accuracy: 0.1795
Epoch 3: saving model to model_init_GRU4_2023-07-0205_46_55.002597/model-00003-1.89505-0.17949-1.84066-0.24000.h5
21/21 [=

## observation
1. Again we have underfitting

## TODO
1. We need to increase complexity of model

## Hypothesis
1. Adding one more CNN layer will reduce underfitting


## Experiment 9: added CNN layer

In [90]:
model8 = Sequential()
model8.add(TimeDistributed(Conv2D(32, (3,3), activation='relu'), input_shape=input_shape))
model8.add(TimeDistributed(MaxPooling2D((2,2))))
model8.add(Dropout(0.5))

model8.add(TimeDistributed(Conv2D(64, (3,3), activation='relu'), input_shape=input_shape))
model8.add(TimeDistributed(MaxPooling2D((2,2))))
model8.add(Dropout(0.5))

model8.add(TimeDistributed(GlobalAveragePooling2D()))
# model5.add(TimeDistributed(Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01))))
# model5.add(Dropout(0.5))

model8.add(GRU(64,kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model8.add(Dropout(0.2))
model8.add(Dense(5, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(0.01)))

In [91]:
optimiser = optimizers.Adam()#write your optimizer
model8.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model8.summary())

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_82 (TimeDi  (None, 15, 98, 98, 32)   896       
 stributed)                                                      
                                                                 
 time_distributed_83 (TimeDi  (None, 15, 49, 49, 32)   0         
 stributed)                                                      
                                                                 
 dropout_34 (Dropout)        (None, 15, 49, 49, 32)    0         
                                                                 
 time_distributed_84 (TimeDi  (None, 15, 47, 47, 64)   18496     
 stributed)                                                      
                                                                 
 time_distributed_85 (TimeDi  (None, 15, 23, 23, 64)   0         
 stributed)                                          

In [92]:
model_name = 'model_init_GRU5' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [94]:
num_epochs = 20
model8.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-94-7b5f23e0beac>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model8.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/20
21/21 [==============================] - ETA: 0s - loss: 1.3274 - categorical_accuracy: 0.4148
Epoch 1: saving model to model_init_GRU5_2023-07-0205_46_55.002597/model-00001-1.32739-0.41478-1.51510-0.37000.h5
21/21 [==============================] - 36s 2s/step - loss: 1.3274 - categorical_accuracy: 0.4148 - val_loss: 1.5151 - val_categorical_accuracy: 0.3700 - lr: 0.0010
Epoch 2/20
21/21 [==============================] - ETA: 0s - loss: 1.2469 - categorical_accuracy: 0.4585
Epoch 2: saving model to model_init_GRU5_2023-07-0205_46_55.002597/model-00002-1.24687-0.45852-1.61879-0.31000.h5
21/21 [==============================] - 37s 2s/step - loss: 1.2469 - categorical_accuracy: 0.4585 - val_loss: 1.6188 - val_categorical_accuracy: 0.3100 - lr: 0.0010
Epoch 3/20
21/21 [==============================] - ETA: 0s - loss: 1.2754 - categorical_accuracy: 0.4057
Epoch 3: saving model to model_init_GRU5_2023-07-0205_46_55.002597/model-00003-1.27539-0.40573-1.37053-0.42000.h5
21/21 [=

## observation
1. Again we have underfitting

## TODO
1. We need to increase complexity of model

## Hypothesis
1. Replacing GRU with LSTM will reduce underfitting


In [95]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Bidirectional, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D, GlobalAveragePooling3D, ConvLSTM2D
from keras.layers.convolutional import Conv2D, Conv3D, MaxPooling2D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

## Experiment 10: LSTM instead of GRU

In [112]:
model9 = Sequential()
model9.add(TimeDistributed(Conv2D(32, (3,3), activation='relu'), input_shape=input_shape))
model9.add(TimeDistributed(MaxPooling2D((2,2))))
model9.add(Dropout(0.5))
model9.add(TimeDistributed(Conv2D(64, (3,3), activation='relu'), input_shape=input_shape))
model9.add(TimeDistributed(MaxPooling2D((2,2))))
model9.add(Dropout(0.5))
model9.add(ConvLSTM2D(filters=128, kernel_size=(3, 3), activation='relu'))
model9.add(Dropout(0.2))
model9.add(Flatten())
model9.add(Dense(5, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(0.01)))

In [113]:
optimiser = optimizers.Adam()#write your optimizer
model9.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model9.summary())

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_102 (TimeD  (None, 15, 98, 98, 32)   896       
 istributed)                                                     
                                                                 
 time_distributed_103 (TimeD  (None, 15, 49, 49, 32)   0         
 istributed)                                                     
                                                                 
 dropout_46 (Dropout)        (None, 15, 49, 49, 32)    0         
                                                                 
 time_distributed_104 (TimeD  (None, 15, 47, 47, 64)   18496     
 istributed)                                                     
                                                                 
 time_distributed_105 (TimeD  (None, 15, 23, 23, 64)   0         
 istributed)                                         

In [114]:
model_name = 'model_init_LSTM0' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [117]:
num_epochs = 20
history9 = model9.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-117-e7d7b5bd2909>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history9 = model9.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/20
21/21 [==============================] - ETA: 0s - loss: 0.1723 - categorical_accuracy: 0.9864
Epoch 1: saving model to model_init_LSTM0_2023-07-0205_46_55.002597/model-00001-0.17233-0.98643-1.65629-0.62000.h5
21/21 [==============================] - 40s 2s/step - loss: 0.1723 - categorical_accuracy: 0.9864 - val_loss: 1.6563 - val_categorical_accuracy: 0.6200 - lr: 0.0010
Epoch 2/20
21/21 [==============================] - ETA: 0s - loss: 0.1443 - categorical_accuracy: 0.9955
Epoch 2: saving model to model_init_LSTM0_2023-07-0205_46_55.002597/model-00002-0.14433-0.99548-2.02172-0.62000.h5
21/21 [==============================] - 43s 2s/step - loss: 0.1443 - categorical_accuracy: 0.9955 - val_loss: 2.0217 - val_categorical_accuracy: 0.6200 - lr: 0.0010
Epoch 3/20
21/21 [==============================] - ETA: 0s - loss: 0.1286 - categorical_accuracy: 0.9970
Epoch 3: saving model to model_init_LSTM0_2023-07-0205_46_55.002597/model-00003-0.12860-0.99698-1.97752-0.66000.h5
21/21

## observation
1. We have a decent model with 99% train and 71% val accuracy.
2. We still see some overfitting



# DECIDING UPON THE BEST MODEL
Using Conv3D based model, we were able to best achieve 74% val accuracy.

Using LSTM based model, we were able to best achieve 71% val accuracy.

Both are decent results.

But let's compare parameters:
- Conv3D based model Total params: **17,393,797**
- LSTM based model Total params  : **1,186,885**

With just 7% of the parameters used by Conv3D based model, our LSTM based model brings about the same accuracy (71% validation accuracy).

Thus, the LSTM based model would be our final model.